In [1]:
import random
import pandas as pd

# Kano Analysis
## Kano analysis is a framework for prioritizing product features from a backlog
## To conduct a Kano analysis, we need:
- A set of users to survey
- A small set of features to study
- Two questions about each feature:
    - Functional question: How do you feel when Feature X is available?
    - Dysfunctional question: How do you feel when Feature X is unavailable?
- A limited list of valid answers to each question

In [2]:
# simulate 100 users to be surveyed
users = [f'User {i}' for i in range(1, 101)]

# simulate 5 features to be studied
features = [f'Feature {i}' for i in range(1, 6)]

# specify the valid answers for each question
potentialAnswers = ['Like it', 'Expect it', 'Don\'t care', 'Can live with it', 'Dislike it']

In [3]:
# define a weight matrix that approximates the relationship between functional and dysfunctional answers about the same feature
weights = pd.DataFrame(columns = potentialAnswers, index = potentialAnswers)
weights.loc['Like it'] = [0.1, 0.1, 0.1, 0.1, 0.6]
weights.loc['Expect it'] = [0.1, 0.1, 0.1, 0.6, 0.1]
weights.loc['Don\'t care'] = [0.1, 0.1, 0.6, 0.1, 0.1]
weights.loc['Can live with it'] = [0.1, 0.6, 0.1, 0.1, 0.1]
weights.loc['Dislike it'] = [0.6, 0.1, 0.1, 0.1, 0.1]
weights

,Like it,Expect it,Don't care,Can live with it,Dislike it
Like it,0.1,0.1,0.1,0.1,0.6
Expect it,0.1,0.1,0.1,0.6,0.1
Don't care,0.1,0.1,0.6,0.1,0.1
Can live with it,0.1,0.6,0.1,0.1,0.1
Dislike it,0.6,0.1,0.1,0.1,0.1


In [4]:
# simulate a set of answers for each user
answers = pd.DataFrame()
for user in users:
    for feature in features:
        # simulate variation in answer to functional question
        functionalAnswer = random.choice(potentialAnswers)
        
        # expect an inverse response to the dysfunctional question - with some variation
        dysfunctionalAnswer = random.choices(potentialAnswers, weights = weights.loc[functionalAnswer], k = 1)[0]
        
        # combine these elements
        answer = pd.DataFrame([[user, feature, functionalAnswer, dysfunctionalAnswer]]
                              , columns = ['User', 'Feature', 'Functional Answer', 'Dysfunctional Answer'])
        
        # stack responses
        answers = pd.concat([answers, answer], ignore_index = True)
        
# convert answers to ordered categorical features for ease of evaluation
answers['Functional Answer'] = pd.Categorical(answers['Functional Answer']
                                              , categories = potentialAnswers, ordered = True)
answers['Dysfunctional Answer'] = pd.Categorical(answers['Dysfunctional Answer']
                                                 , categories = potentialAnswers, ordered = True)

In [5]:
# verify that answers are sensible
answers.groupby(['Functional Answer', 'Dysfunctional Answer'])['User'].count()

Functional Answer  Dysfunctional Answer
Like it            Like it                  8
                   Expect it                8
                   Don't care               8
                   Can live with it        12
                   Dislike it              72
Expect it          Like it                  3
                   Expect it                7
                   Don't care               9
                   Can live with it        73
                   Dislike it               7
Don't care         Like it                  5
                   Expect it                8
                   Don't care              66
                   Can live with it        13
                   Dislike it              11
Can live with it   Like it                  7
                   Expect it               67
                   Don't care              10
                   Can live with it         9
                   Dislike it               9
Dislike it         Like it              